# **Homework 7 - Bert (Question Answering)**

If you have any questions, feel free to email us at ntu-ml-2023spring-ta@googlegroups.com



Slide:    [Link](https://docs.google.com/presentation/d/15lGUmT8NpLGtoxRllRWCJyQEjhR1Idcei63YHsDckPE/edit#slide=id.g21fff4e9af6_0_13)　Kaggle: [Link](https://www.kaggle.com/competitions/ml2023spring-hw7/host/sandbox-submissions)　Data: [Link](https://drive.google.com/file/d/1YU9KZFhQqW92Lw9nNtuUPg0-8uyxluZ7/view?usp=sharing)




# Prerequisites

## Download Dataset

In [5]:
pip install gdown

Note: you may need to restart the kernel to use updated packages.


In [6]:
# download dataset by youself!
!gdown --id '1TjoBdNlGBhP_J9C66MOY7ILIrydm7ZCS' --output hw7_data.zip
# unzip dataset
!unzip -o hw7_data.zip

/opt/conda/lib/python3.7/site-packages/gdown/cli.py:130: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  category=FutureWarning,
Downloading...
From: https://drive.google.com/uc?id=1TjoBdNlGBhP_J9C66MOY7ILIrydm7ZCS
To: /kaggle/working/hw7_data.zip
100%|██████████████████████████████████████| 12.1M/12.1M [00:00<00:00, 79.8MB/s]
Archive:  hw7_data.zip
  inflating: hw7_train.json          
  inflating: hw7_test.json           
  inflating: hw7_dev.json            
  inflating: hw7_in-context-learning-examples.json  


## Install packages

Documentation for the toolkit:　https://huggingface.co/transformers/

In [7]:
# You are allowed to change version of transformers or use other toolkits
!pip install transformers==4.26.1
!pip install accelerate==0.16.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.7/199.7 kB 7.5 MB/s eta 0:00:00
  Attempting uninstall: accelerate
    Found existing installation: accelerate 0.12.0
    Uninstalling accelerate-0.12.0:
      Successfully uninstalled accelerate-0.12.0


# Kaggle (Fine-tuning)

## Task description
- Chinese Extractive Question Answering
  - Input: Paragraph + Question
  - Output: Answer

- Objective: Learn how to fine tune a pretrained model on downstream task using transformers

- Todo
    - Fine tune a pretrained chinese BERT model
    - Change hyperparameters (e.g. doc_stride)
    - Apply linear learning rate decay
    - Try other pretrained models
    - Improve preprocessing
    - Improve postprocessing
- Training tips
    - Automatic mixed precision
    - Gradient accumulation
    - Ensemble

- Estimated training time (tesla t4 with automatic mixed precision enabled)
    - Simple: 8mins
    - Medium: 8mins
    - Strong: 25mins
    - Boss: 2hrs
  

## Import Packages

In [24]:
import json
import numpy as np
import random
import torch
from torch.utils.data import DataLoader, Dataset 
from transformers import AdamW, BertForQuestionAnswering, BertTokenizerFast, get_linear_schedule_with_warmup

from tqdm.auto import tqdm

device = "cuda" if torch.cuda.is_available() else "cpu"

# Fix random seed for reproducibility
def same_seeds(seed):
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)
    np.random.seed(seed)
    random.seed(seed)
    torch.backends.cudnn.benchmark = False
    torch.backends.cudnn.deterministic = True
same_seeds(2)

## Load Model and Tokenizer




 

In [9]:
from transformers import (BertTokenizerFast,AutoModelForQuestionAnswering,BertForQuestionAnswering, BertTokenizerFast)
model = BertForQuestionAnswering.from_pretrained("luhua/chinese_pretrain_mrc_roberta_wwm_ext_large").to(device)
tokenizer = BertTokenizerFast.from_pretrained("luhua/chinese_pretrain_mrc_roberta_wwm_ext_large")
#reference : https://huggingface.co/luhua/chinese_pretrain_mrc_roberta_wwm_ext_large
# You can safely ignore the warning message (it pops up because new prediction heads for QA are initialized randomly)

## Read Data

- Training set: 26918 QA pairs
- Dev set: 2863  QA pairs
- Test set: 3524  QA pairs

- {train/dev/test}_questions:	
  - List of dicts with the following keys:
   - id (int)
   - paragraph_id (int)
   - question_text (string)
   - answer_text (string)
   - answer_start (int)
   - answer_end (int)
- {train/dev/test}_paragraphs: 
  - List of strings
  - paragraph_ids in questions correspond to indexs in paragraphs
  - A paragraph may be used by several questions 

In [10]:
def read_data(file):
    with open(file, 'r', encoding="utf-8") as reader:
        data = json.load(reader)
    return data["questions"], data["paragraphs"]

# Change the path of the dataset
train_questions, train_paragraphs = read_data("/kaggle/input/2023-ml-hw7-question-answering/hw7_train.json")
dev_questions, dev_paragraphs = read_data("/kaggle/input/2023-ml-hw7-question-answering/hw7_dev.json")
test_questions, test_paragraphs = read_data("/kaggle/input/2023-ml-hw7-question-answering/hw7_test.json")

## Tokenize Data

In [11]:
# Tokenize questions and paragraphs separately
# 「add_special_tokens」 is set to False since special tokens will be added when tokenized questions and paragraphs are combined in datset __getitem__ 

train_questions_tokenized = tokenizer([train_question["question_text"] for train_question in train_questions], add_special_tokens=False)
dev_questions_tokenized = tokenizer([dev_question["question_text"] for dev_question in dev_questions], add_special_tokens=False)
test_questions_tokenized = tokenizer([test_question["question_text"] for test_question in test_questions], add_special_tokens=False) 

train_paragraphs_tokenized = tokenizer(train_paragraphs, add_special_tokens=False)
dev_paragraphs_tokenized = tokenizer(dev_paragraphs, add_special_tokens=False)
test_paragraphs_tokenized = tokenizer(test_paragraphs, add_special_tokens=False, max_length=512)

# You can safely ignore the warning message as tokenized sequences will be futher processed in datset __getitem__ before passing to model

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


## Dataset

In [12]:
class QA_Dataset(Dataset):
    def __init__(self, split, questions, tokenized_questions, tokenized_paragraphs):
        self.split = split
        self.questions = questions
        self.tokenized_questions = tokenized_questions
        self.tokenized_paragraphs = tokenized_paragraphs
        self.max_question_len = 50
        self.max_paragraph_len = 400
        
        ##### TODO: Change value of doc_stride #####
        self.doc_stride = 300

        # Input sequence length = [CLS] + question + [SEP] + paragraph + [SEP]
        self.max_seq_len = 1 + self.max_question_len + 1 + self.max_paragraph_len + 1

    def __len__(self):
        return len(self.questions)

    def __getitem__(self, idx):
        question = self.questions[idx]
        tokenized_question = self.tokenized_questions[idx]
        tokenized_paragraph = self.tokenized_paragraphs[question["paragraph_id"]]

        ##### TODO: trainprocessing #####
        # Hint: How to prevent model from learning something it should not learn
        if self.split == "train":
            # Convert answer's start/end positions in paragraph_text to start/end positions in tokenized_paragraph  
            answer_start_token = tokenized_paragraph.char_to_token(question["answer_start"])
            answer_end_token = tokenized_paragraph.char_to_token(question["answer_end"])


            # A single window is obtained by slicing the portion of paragraph containing the answer
            mid = (answer_start_token + answer_end_token) // 2
            paragraph_start = max(0, min(mid - self.max_paragraph_len // 2, len(tokenized_paragraph) - self.max_paragraph_len))
            paragraph_end = paragraph_start + self.max_paragraph_len

            # Slice question/paragraph and add special tokens (101: CLS, 102: SEP)
            input_ids_question = [101] + tokenized_question.ids[:self.max_question_len] + [102] 
            input_ids_paragraph = tokenized_paragraph.ids[paragraph_start : paragraph_end] + [102]		
            
            # Convert answer's start/end positions in tokenized_paragraph to start/end positions in the window  
            answer_start_token += len(input_ids_question) - paragraph_start
            answer_end_token += len(input_ids_question) - paragraph_start
            
            # Pad sequence and obtain inputs to model 
            input_ids, token_type_ids, attention_mask = self.padding(input_ids_question, input_ids_paragraph)
            return torch.tensor(input_ids), torch.tensor(token_type_ids), torch.tensor(attention_mask), answer_start_token, answer_end_token

        # Validation/Testing
        else:
            input_ids_list, token_type_ids_list, attention_mask_list = [], [], []
            
            # Paragraph is split into several windows, each with start positions separated by step "doc_stride"
            for i in range(0, len(tokenized_paragraph), self.doc_stride):
                
                # Slice question/paragraph and add special tokens (101: CLS, 102: SEP)
                input_ids_question = [101] + tokenized_question.ids[:self.max_question_len] + [102]
                input_ids_paragraph = tokenized_paragraph.ids[i : i + self.max_paragraph_len] + [102]
                
                # Pad sequence and obtain inputs to model
                input_ids, token_type_ids, attention_mask = self.padding(input_ids_question, input_ids_paragraph)
                
                input_ids_list.append(input_ids)
                token_type_ids_list.append(token_type_ids)
                attention_mask_list.append(attention_mask)
            
            return torch.tensor(input_ids_list), torch.tensor(token_type_ids_list), torch.tensor(attention_mask_list)

    def padding(self, input_ids_question, input_ids_paragraph):
        # Pad zeros if sequence length is shorter than max_seq_len
        padding_len = self.max_seq_len - len(input_ids_question) - len(input_ids_paragraph)
        # Indices of input sequence tokens in the vocabulary
        input_ids = input_ids_question + input_ids_paragraph + [0] * padding_len
        # Segment token indices to indicate first and second portions of the inputs. Indices are selected in [0, 1]
        token_type_ids = [0] * len(input_ids_question) + [1] * len(input_ids_paragraph) + [0] * padding_len
        # Mask to avoid performing attention on padding token indices. Mask values selected in [0, 1]
        attention_mask = [1] * (len(input_ids_question) + len(input_ids_paragraph)) + [0] * padding_len
        
        return input_ids, token_type_ids, attention_mask

train_set = QA_Dataset("train", train_questions, train_questions_tokenized, train_paragraphs_tokenized)
dev_set = QA_Dataset("dev", dev_questions, dev_questions_tokenized, dev_paragraphs_tokenized)
test_set = QA_Dataset("test", test_questions, test_questions_tokenized, test_paragraphs_tokenized)

## Function for Evaluation

In [13]:
def evaluate(data, output):
    ##### TODO: Postprocessing #####
    # There is a bug and room for improvement in postprocessing 
    # Hint: Open your prediction file to see what is wrong 
    
    answer = ''
    max_prob = float('-inf')
    num_of_windows = data[0].shape[1]
    
    for k in range(num_of_windows):
        # Obtain answer by choosing the most probable start position / end position
        start_prob, start_index = torch.max(output.start_logits[k], dim=0)
        end_prob, end_index = torch.max(output.end_logits[k], dim=0)
        #認為該window中不存在合法的答案，跳過該window。
        if start_index<=end_index:
            pass
        else:
            continue
        
        
        # Probability of answer is calculated as sum of start_prob and end_prob
        prob = start_prob + end_prob
        
        # Replace answer if calculated probability is larger than previous windows
        #將prob與先前窗口的答案概率max_prob進行比較，如果prob比max_prob大，則更新max_prob和答案文本answer。
        if prob > max_prob:
            max_prob = prob
            # Convert tokens to chars (e.g. [1920, 7032] --> "大 金")
            answer = tokenizer.decode(data[0][0][k][start_index : end_index + 1])
    #在tokenizer.encode過程中，可能會在詞與詞之間加上空格，所以這裡使用replace函數刪除答案文本中的空格。
    # Remove spaces in answer (e.g. "大 金" --> "大金")
    return answer.replace(' ','')

In [22]:
train_batch_size = 1
# dataloader
# Note: Do NOT change batch size of dev_loader / test_loader !
# Although batch size=1, it is actually a batch consisting of several windows from the same QA pair
train_loader = DataLoader(train_set, batch_size=train_batch_size, shuffle=True, pin_memory=True)
dev_loader = DataLoader(dev_set, batch_size=1, shuffle=False, pin_memory=True)
test_loader = DataLoader(test_set, batch_size=1, shuffle=False, pin_memory=True)

## Training

In [27]:
from accelerate import Accelerator
# hyperparameters
num_epoch = 2
validation = True
logging_step = 100
learning_rate = 1e-5
accumulate_iter = 4
optimizer = AdamW(model.parameters(), lr=learning_rate)
#reference : https://huggingface.co/docs/transformers/main_classes/optimizer_schedules
#用來設定optimizer的learning rate schedule，使用的函式為get_linear_schedule_with_warmup()，是hugging face提供的內建函式之一。
updates = num_epoch * len(train_loader) // accumulate_iter + num_epoch
schedular = get_linear_schedule_with_warmup(optimizer, num_warmup_steps = 0.1*updates , num_training_steps = updates)


#### TODO: gradient_accumulation (optional)####
# Note: train_batch_size * gradient_accumulation_steps = effective batch size
# If CUDA out of memory, you can lower the train_batch_size and upper the gradient_accumulation_steps
# Doc: https://huggingface.co/docs/accelerate/usage_guides/gradient_accumulation
gradient_accumulation_steps = 16


# Change "fp16_training" to True to support automatic mixed 
# precision training (fp16)	
fp16_training = True
if fp16_training: 
    !pip install accelerate==0.16.0
    from accelerate import Accelerator
    accelerator = Accelerator(mixed_precision='fp16')
    device = accelerator.device


# Documentation for the toolkit:  https://huggingface.co/docs/accelerate/
if fp16_training: 
    model, optimizer, train_loader = accelerator.prepare(model, optimizer, train_loader) 
model.train()


print("Start Training ...")

for epoch in range(num_epoch):
    step = 1
    train_loss = train_acc = 0
    
    for batch_index,data in enumerate(tqdm(train_loader)):
        # Load all data into GPU
        data = [i.to(device) for i in data]
        
        # Model inputs: input_ids, token_type_ids, attention_mask, start_positions, end_positions (Note: only "input_ids" is mandatory)
        # Model outputs: start_logits, end_logits, loss (return when start_positions/end_positions are provided)  
        output = model(input_ids=data[0], token_type_ids=data[1], attention_mask=data[2], start_positions=data[3], end_positions=data[4])
        # Choose the most probable start position / end position
        start_index = torch.argmax(output.start_logits, dim=1)
        end_index = torch.argmax(output.end_logits, dim=1)
        
        # Prediction is correct only if both start_index and end_index are correct
        train_acc += ((start_index == data[3]) & (end_index == data[4])).float().mean()
           
        train_loss += output.loss
        
        if fp16_training:
            norm_loss = output.loss / accumulate_iter
            accelerator.backward(norm_loss)
        else:
            output.loss.backward()
        lenght = batch_index+1
        
        if (lenght % accumulate_iter == 0) or (lenght == len(train_loader)):
            optimizer.step()
            optimizer.zero_grad()
            schedular.step()
        
        step += 1
        ##### TODO: Apply linear learning rate decay #####

        # Print training loss and accuracy over past logging step
        if step % logging_step == 0:
            print(f"Epoch {epoch + 1} | Step {step} | loss = {train_loss.item() / logging_step:.3f}, acc = {train_acc / logging_step:.3f}")
            train_loss = train_acc = 0

    if validation:
        print("Evaluating Dev Set ...")
        model.eval()
        with torch.no_grad():
            dev_acc = 0
            for i, data in enumerate(tqdm(dev_loader)):
                output = model(input_ids=data[0].squeeze(dim=0).to(device), token_type_ids=data[1].squeeze(dim=0).to(device),
                       attention_mask=data[2].squeeze(dim=0).to(device))
                # prediction is correct only if answer text exactly matches
                dev_acc += evaluate(data, output) == dev_questions[i]["answer_text"]
            print(f"Validation | Epoch {epoch + 1} | acc = {dev_acc / len(dev_loader):.3f}")
        model.train()

# Save a model and its configuration file to the directory 「saved_model」 
# i.e. there are two files under the direcory 「saved_model」: 「pytorch_model.bin」 and 「config.json」
# Saved model can be re-loaded using 「model = BertForQuestionAnswering.from_pretrained("saved_model")」
print("Saving Model ...")
model_save_dir = "saved_model" 
model.save_pretrained(model_save_dir)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
Start Training ...


  0%|          | 0/26918 [00:00<?, ?it/s]

Epoch 1 | Step 100 | loss = 2.796, acc = 0.350
Epoch 1 | Step 200 | loss = 2.309, acc = 0.480
Epoch 1 | Step 300 | loss = 2.456, acc = 0.420
Epoch 1 | Step 400 | loss = 1.696, acc = 0.490
Epoch 1 | Step 500 | loss = 1.620, acc = 0.510
Epoch 1 | Step 600 | loss = 1.401, acc = 0.520
Epoch 1 | Step 700 | loss = 1.171, acc = 0.590
Epoch 1 | Step 800 | loss = 1.149, acc = 0.610
Epoch 1 | Step 900 | loss = 1.081, acc = 0.600
Epoch 1 | Step 1000 | loss = 1.031, acc = 0.590
Epoch 1 | Step 1100 | loss = 0.795, acc = 0.700
Epoch 1 | Step 1200 | loss = 1.131, acc = 0.600
Epoch 1 | Step 1300 | loss = 0.826, acc = 0.690
Epoch 1 | Step 1400 | loss = 0.839, acc = 0.660
Epoch 1 | Step 1500 | loss = 0.856, acc = 0.670
Epoch 1 | Step 1600 | loss = 0.758, acc = 0.690
Epoch 1 | Step 1700 | loss = 0.777, acc = 0.690
Epoch 1 | Step 1800 | loss = 0.673, acc = 0.750
Epoch 1 | Step 1900 | loss = 1.071, acc = 0.620
Epoch 1 | Step 2000 | loss = 0.588, acc = 0.780
Epoch 1 | Step 2100 | loss = 0.934, acc = 0.610
E

  0%|          | 0/2863 [00:00<?, ?it/s]

Validation | Epoch 1 | acc = 0.855


  0%|          | 0/26918 [00:00<?, ?it/s]

Epoch 2 | Step 100 | loss = 0.431, acc = 0.820
Epoch 2 | Step 200 | loss = 0.434, acc = 0.840
Epoch 2 | Step 300 | loss = 0.341, acc = 0.810
Epoch 2 | Step 400 | loss = 0.351, acc = 0.850
Epoch 2 | Step 500 | loss = 0.247, acc = 0.900


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>                                                                                      │
│                                                                                                  │
│   59 │   │                                                                                       │
│   60 │   │   if fp16_training:                                                                   │
│   61 │   │   │   norm_loss = output.loss / accumulate_iter                                       │
│ ❱ 62 │   │   │   accelerator.backward(norm_loss)                                                 │
│   63 │   │   else:                                                                               │
│   64 │   │   │   output.loss.backward()                                                          │
│   65 │   │   lenght = batch_index+1                                                              │
│                                                                                                  │
│ /opt/conda/lib/python3.7/site-packages/accelerate/accelerator.py:1439 in backward                │
│                                                                                                  │
│   1436 │   │   elif self.distributed_type == DistributedType.MEGATRON_LM:                        │
│   1437 │   │   │   return                                                                        │
│   1438 │   │   elif self.scaler is not None:                                                     │
│ ❱ 1439 │   │   │   self.scaler.scale(loss).backward(**kwargs)                                    │
│   1440 │   │   else:                                                                             │
│   1441 │   │   │   loss.backward(**kwargs)                                                       │
│   1442                                                                                           │
│                                                                                                  │
│ /opt/conda/lib/python3.7/site-packages/torch/_tensor.py:488 in backward                          │
│                                                                                                  │
│    485 │   │   │   │   inputs=inputs,                                                            │
│    486 │   │   │   )                                                                             │
│    487 │   │   torch.autograd.backward(                                                          │
│ ❱  488 │   │   │   self, gradient, retain_graph, create_graph, inputs=inputs                     │
│    489 │   │   )                                                                                 │
│    490 │                                                                                         │
│    491 │   def register_hook(self, hook):                                                        │
│                                                                                                  │
│ /opt/conda/lib/python3.7/site-packages/torch/autograd/__init__.py:199 in backward                │
│                                                                                                  │
│   196 │   # calls in the traceback and some print out the last line                              │
│   197 │   Variable._execution_engine.run_backward(  # Calls into the C++ engine to run the bac   │
│   198 │   │   tensors, grad_tensors_, retain_graph, create_graph, inputs,                        │
│ ❱ 199 │   │   allow_unreachable=True, accumulate_grad=True)  # Calls into the C++ engine to ru   │
│   200                                                                                            │
│   201 def grad(                                                                                  │
│   202 │   outputs: _TensorOrTensors,                       

## Testing

In [28]:
print("Evaluating Test Set ...")

result = []

model.eval()
with torch.no_grad():
    for data in tqdm(test_loader):
        output = model(input_ids=data[0].squeeze(dim=0).to(device), token_type_ids=data[1].squeeze(dim=0).to(device),
                       attention_mask=data[2].squeeze(dim=0).to(device))
        result.append(evaluate(data, output))

result_file = "result.csv"
with open(result_file, 'w') as f:
    f.write("ID,Answer\n")
    for i, test_question in enumerate(test_questions):
    # Replace commas in answers with empty strings (since csv is separated by comma)
    # Answers in kaggle are processed in the same way
        f.write(f"{test_question['id']},{result[i].replace(',','')}\n")

print(f"Completed! Result is in {result_file}")

Evaluating Test Set ...


  0%|          | 0/3524 [00:00<?, ?it/s]

Completed! Result is in result.csv


# GradeScope - Question 2 (In-context learning)

### Try in-context learning
The example prompt is :
```
請從最後一篇的文章中找出最後一個問題的答案：
文章：<文章內容>
問題：<問題敘述>
答案：<答案>
```

In [30]:
import torch

# To avoid CUDA_OUT_OF_MEMORY
torch.set_default_tensor_type(torch.cuda.FloatTensor)

# Fix random seed for reproducibility
def same_seeds(seed):
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)
    np.random.seed(seed)
    random.seed(seed)
    torch.backends.cudnn.benchmark = False
    torch.backends.cudnn.deterministic = True
same_seeds(2)

In [31]:
from transformers import AutoTokenizer, AutoModelForCausalLM

# You can try model with different size
# When using Colab or Kaggle, model with more than 2 billions parameters may 
# run out of memory
tokenizer = AutoTokenizer.from_pretrained("facebook/xglm-1.7B")
model = AutoModelForCausalLM.from_pretrained("facebook/xglm-1.7B")

In [32]:
# for cleaning model output. If you try different prompts, you may have to fix 
# this function on your own
def clean_text(text):
    # Note: When you use unilingual model, the colon may become fullwidth
    text = text.split("答案:")[-1]
    text = text.split(" ")[0]
    return text

In [39]:
import random
import json

with open("hw7_in-context-learning-examples.json", "r") as f: 
    test = json.load(f)

# K-shot learning 
# Give model K examples to make it achieve better accuracy 
# Note: (1) When K >= 4, CUDA_OUT_OFF_MEMORY may occur.
#       (2) The maximum input length of XGLM is 2048
K = 2

question_ids = [qa["id"] for qa in test["questions"]]

with open("in-context-learning-result.txt", "w") as f:
    print("ID,Ground-Truth,Prediction", file = f)
    with torch.no_grad():
        for idx, qa in enumerate(test["questions"]):
            # You can try different prompts
            prompt = "請仔細根據下面的文章回答問題\n"
            exist_question_indexs = [question_ids.index(qa["id"])]

            # K-shot learning: give the model K examples with answers
            for i in range(K):
                question_index = question_ids.index(qa["id"])
                while(question_index in exist_question_indexs): 
                    question_index = random.randint(0, len(question_ids) - 1)
                exist_question_indexs.append(question_index)    
                paragraph_id = test["questions"][question_index]["paragraph_id"]
                prompt += f'文章：{test["paragraphs"][paragraph_id]}\n'
                prompt += f'問題：{test["questions"][question_index]["question_text"]}\n'
                prompt += f'答案：{test["questions"][question_index]["answer_text"]}\n'

            # The final one question without answer
            paragraph_id = qa["paragraph_id"]
            prompt += f'文章：{test["paragraphs"][paragraph_id]}\n'
            prompt += f'問題：{qa["question_text"]}\n'
            prompt += f'答案：'
            
            inputs = tokenizer(prompt, add_special_tokens=False, return_tensors="pt") 
            sample = model.generate(**inputs, max_new_tokens = 20)
            text = tokenizer.decode(sample[0], skip_special_tokens=True)

            # Note: You can delete this line to see what will happen
            text = clean_text(text)
            
            print(prompt)
            print(f'正確答案: {qa["answer_text"]}')
            print(f'模型輸出: {text}')
            print()

            print(f"{idx},{qa['answer_text']},{text}", file = f)

請仔細根據下面的文章回答問題
文章：2010年7月，1000多名市民聚集在地鐵江南西站出口處舉行2010年廣州支持粵語行動請願活動。在收到有關該活動被取消的消息後，他們仍然按計劃時間到達為光復文化發行自己的活動。語音。在內地媒體封鎖消息傳出後，廣州還封鎖了與香港電視有關的新聞片段。8月1日，廣州中央公園的粵語聚會受到當局的強烈阻撓。人們被迫轉移路線。當他們到達北京路南段時，他們被大批警察包圍。參與者被帶到公共汽車並在東山體育場被帶走。它在被質疑和被定罪後被釋放。大陸請願制度允許人們向上級政府部門發出呼籲。廣東省政府經常邀請來自省內各地的請願人來到東風路正門，得到省政府的幫助。在集會時，當局將調動大量警察部隊積極處理他們。在對峙期間，交通通常會很慢，但當地官方媒體從未報導相關報導。2014年初，更重要的請願者是該省的家禽商，由於市場被迫關閉，在禽流感期間遭受了損失，以及隋毛反對茂名石化項目的年輕人。
問題：2010年廣州撐粵語行動請願活動是在當年的什麼時候舉行的？
答案：7月
文章：廣州是京廣鐵路，廣深鐵路，廣茂鐵路和廣梅鐵路的終點站。2009年底，武廣客運專線投入運營，多機組列車長980公里，最高時速350公里。2011年1月7日，廣珠城際鐵路投入運營，平均時速200公里。廣州鐵路，長途巴士和渡輪直達香港。廣九快速列車從廣州火車東站出發，直達香港紅磡火車站。總長約182公里，行程大約需要兩個小時。每年都有繁忙的教練從香港的不同乘客點接載乘客。在市中心的珠江北岸有一條渡輪線路，河流居民可以直接過河而無需乘坐公共汽車或步行穿過大橋。每天都有往返南沙碼頭和蓮花山碼頭的高速雙體船。渡輪也開往香港中國客運碼頭和港澳客運碼頭。
問題：廣珠城際鐵路平均每小時可以走多遠？
答案：200公里
文章：2010年引入的廣州快速交通運輸系統是世界第二大快速運輸系統。每日載客量可達100萬人次。每小時的客流量峰值高達26,900名乘客，僅次於波哥大的快速交通系統。每10秒有一輛公共汽車，每輛公共汽車在一個方向上行駛350小時。該平台包括橋樑，是世界上最長的國家公共汽車快速運輸系統平台，長度為260米。目前，廣州市的出租車和公交車主要以液化石油氣為燃料，部分公交車採用油電，氣電混合技術。2012年底，一輛LNG燃料公共汽車開始啟動。2014年6月，引入了LNG插電式混合動力公交車取代LPG公交

In [41]:
import random
import json
# Change the path of you data
with open("/kaggle/input/2023-ml-hw7-question-answering/hw7_in-context-learning-examples.json", "r") as f: 
    test = json.load(f)

# K-shot learning 
# Give model K examples to make it achieve better accuracy 
# Note: When K >= 4, CUDA_OUT_OFF_MEMORY may occur
K = 1


question_ids = [qa["id"] for qa in test["questions"]]

with torch.no_grad():
    for idx, qa in enumerate(test["questions"]):
        # You can try different prompts
        prompt = "拜託根據下面的文章回答問題\n"
        exist_question_indexs = [question_ids.index(qa["id"])]
        # K-shot learning: give the model K examples with answers
        for i in range(K):
            question_index = question_ids.index(qa["id"])
            while(question_index in exist_question_indexs): 
                question_index = random.randint(0, len(question_ids) - 1)
            exist_question_indexs.append(question_index)    
            paragraph_id = test["questions"][question_index]["paragraph_id"]
            prompt += f'文章：{test["paragraphs"][paragraph_id]}\n'
            prompt += f'問題：{test["questions"][question_index]["question_text"]}\n'
            prompt += f'答案：{test["questions"][question_index]["answer_text"]}\n'


        # The final one question without answer
        paragraph_id = qa["paragraph_id"]
        prompt += f'文章：{test["paragraphs"][paragraph_id]}\n'
        prompt += f'問題：{qa["question_text"]}\n'
        prompt += f'答案：'
        
        inputs = tokenizer(prompt, add_special_tokens=False, return_tensors="pt") 
        sample = model.generate(**inputs, max_new_tokens = 20)
        text = tokenizer.decode(sample[0], skip_special_tokens=True)
        # Note: You can delete this line to see what will happen
        text = clean_text(text)
        
        print(prompt)
        print(f'正確答案: {qa["answer_text"]}')
        print(f'模型輸出: {text}')
        print()


拜託根據下面的文章回答問題
文章：2010年7月，1000多名市民聚集在地鐵江南西站出口處舉行2010年廣州支持粵語行動請願活動。在收到有關該活動被取消的消息後，他們仍然按計劃時間到達為光復文化發行自己的活動。語音。在內地媒體封鎖消息傳出後，廣州還封鎖了與香港電視有關的新聞片段。8月1日，廣州中央公園的粵語聚會受到當局的強烈阻撓。人們被迫轉移路線。當他們到達北京路南段時，他們被大批警察包圍。參與者被帶到公共汽車並在東山體育場被帶走。它在被質疑和被定罪後被釋放。大陸請願制度允許人們向上級政府部門發出呼籲。廣東省政府經常邀請來自省內各地的請願人來到東風路正門，得到省政府的幫助。在集會時，當局將調動大量警察部隊積極處理他們。在對峙期間，交通通常會很慢，但當地官方媒體從未報導相關報導。2014年初，更重要的請願者是該省的家禽商，由於市場被迫關閉，在禽流感期間遭受了損失，以及隋毛反對茂名石化項目的年輕人。
問題：2010年廣州撐粵語行動請願活動是在當年的什麼時候舉行的？
答案：7月
文章：2010年引入的廣州快速交通運輸系統是世界第二大快速運輸系統。每日載客量可達100萬人次。每小時的客流量峰值高達26,900名乘客，僅次於波哥大的快速交通系統。每10秒有一輛公共汽車，每輛公共汽車在一個方向上行駛350小時。該平台包括橋樑，是世界上最長的國家公共汽車快速運輸系統平台，長度為260米。目前，廣州市的出租車和公交車主要以液化石油氣為燃料，部分公交車採用油電，氣電混合技術。2012年底，一輛LNG燃料公共汽車開始啟動。2014年6月，引入了LNG插電式混合動力公交車取代LPG公交車。2007年1月16日，廣州市政府完全禁止在城市地區駕駛摩托車。違反禁令的機動車將被沒收。廣州市交通局聲稱，禁令的實施導致交通擁堵和車禍大大減少。廣州白雲國際機場位於白雲區與花都區交界處。它於2004年8月5日正式投入運營。它是中國第二繁忙的機場。機場取代了原先位於市中心的舊機場，無法滿足日益增長的航空需求。目前，機場有三個簡易機場，是中國第三個擁有三條跑道的民航機場。比2023年香港國際機場第三條跑道的預計完工時間提前了8年。
問題：從哪一天開始在廣州市內騎摩托車會被沒收？
答案：
正確答案: 2007年1月16日
模型輸出: 2007年1月16日

拜託根據下面的文章回答問題
文章：作為現代革命的發祥